[Darvas Box Traps Elusive Returns](https://www.investopedia.com/articles/trading/08/darvas.asp)

The rules can be explained so that modern tools like scanning software can identify trading candidates. To quantify the box, traders should look for stocks in which the difference between the high and the low price over **the past four weeks is less than 10% of the stock's high during that time**. As a formula, it can be written as:



```(100 * ((High – Low) / High)) < 10```

Traders can use a larger percentage to get more stocks on their potential buy lists. The buy should be taken at the market's open the morning after the stock closes outside the box by **at least half a point on a volume that is greater than the average 30-day volume**. The initial stop should be set a quarter point below the lowest price of the box. It should be raised as new boxes form, always a quarter point below the low.

---

box に入っている銘柄とは；
+ 過去4週間のHLの差をHで割ったものが0.1以下

box を抜けるとは，
+ 終値がボックスの外に，50％突き抜ける
+ 30日の出来高よりも多くの出来高がある

Entry
+ boxを抜けた次の日の朝


# まずは，dukという銘柄だけでやってみる

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division 

from quantopian.pipeline import Pipeline,CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters.morningstar import Q1500US,Q500US
from quantopian.pipeline.experimental import QTradableStocksUS

from quantopian.pipeline.factors import DailyReturns, SimpleMovingAverage
import alphalens


#duk = get_pricing("duk","2007-1-1", "2017-12-12")


In [ ]:

duk["return"] = duk.price.pct_change()
duk["M20_high"] = duk.high.rolling(20).max()
duk["M20_low"] = duk.high.rolling(20).min()
duk["MA30_vol"] = duk.volume.rolling(30).mean()
duk["high_low_ratio"] = (duk["M20_high"] - duk["M20_low"]) / duk["M20_high"]
duk["volume_ratio"] = duk["volume"] / duk["MA30_vol"] -1

duk["in_the_box"] =  duk["high_low_ratio"] < 0.05
duk["volume_enough"] = duk["volume_ratio"] > 0.5
duk["M20_in_the_box"] = duk.in_the_box.rolling(20).sum()
duk["breakout_box"] = (duk["M20_in_the_box"].shift(1) == 20) & (duk["M20_in_the_box"] != 20)


In [ ]:
duk["intraday_return"] = duk.close_price / duk.open_price - 1
duk["5days_return"] = duk.close_price.shift(-5) / duk.open_price - 1

duk["prev_break"] = duk.breakout_box.shift(1).replace(np.nan, False)
duk["prev_volume_enough"] = duk["volume_enough"].shift(1).replace(np.nan, False)
duk["prev_return"] = duk["return"].shift(1).replace(np.nan, 0) > 0

In [ ]:

import matplotlib.pyplot as plt 

x = "2008-2-23"
plt.plot(duk[:x].price)
plt.plot(duk[:x].in_the_box.rolling(20).sum())
plt.plot(duk[:x].breakout_box)

## 20ではなくなったタイミングで、↑に抜けたか下に抜けたが、それでトレードシグナルとすればよい？

In [ ]:
class HighLowRatio(CustomFactor):
    inputs = [ USEquityPricing.high, USEquityPricing.low]
    
    def compute(self, today, assets, out, high, low):
        highest = np.nanmax(high,axis=0)
        loweset = np.nanmin(low,axis=0)
        high_low_ratio = (highest - loweset)/ highest 
        
        out[:] = high_low_ratio #np.log(high_low_ratio)

class InBox(CustomFactor):
    inputs = [ USEquityPricing.high, USEquityPricing.low]
    window_length = 20
    def compute(self, today, assets, out, high, low):
        high_row_ratio = (high - low) / high
        inthebox = (high_row_ratio < 0.05)
        out[:] = np.sum(inthebox, axis=0)

class MAVolume(CustomFactor):
    inputs = [USEquityPricing.volume]
    window_length = 21 
    def computer(self, today, assets, out, volume):
        #out[:] = 
        out[:] = volume[-1] / volume[:-1].mean()
        
class DarvasBoxBreakUp(CustomFactor):
    inputs = [ USEquityPricing.high, USEquityPricing.low, USEquityPricing.close, USEquityPricing.volume]
    window_length = 21
    def compute(self, today, assets, out, high, low, close, volume):

        high_row_ratio = (high - low) / high
        inthebox = (high_row_ratio < 0.05)
        vol_ma = volume[:-1].mean()
        volume_enough = volume[-1] > vol_ma * 1.5

        
        box_until_prevday = np.sum(inthebox[:-1], axis=0)
        box_until_today  = np.sum(inthebox[1:], axis=0)
        
        today_return = close[-1] / close[-2] - 1
        breakbox = (box_until_prevday == 20) & (box_until_today != 20) & volume_enough
        breakup = today_return > 0.0
        out[:] = breakbox & breakup


In [ ]:

pipe = Pipeline()

pipe.set_screen(Q500US())
pipe.add(SimpleMovingAverage(), "MAVolume")
pipe.add(HighLowRatio(), "HighLowRatio")
pipe.add(InBox(), "InBox")
pipe.add(DarvasBoxBreakUp(), "DarvasBoxBreakUp")
pipe.add(DailyReturns(), "DailyReturns")



start = pd.datetime(2017,1,1)
end = pd.datetime(2018,1,1)
results = run_pipeline(pipe, start_date= start, end_date=end, chunksize=None)


In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.hist(results["HighLowRatio"], bins=100, alpha=0.5, )


In [ ]:
len(results[results["DarvasBoxBreakUp"] ==1])


In [ ]:
results[results["DarvasBoxBreakUp"] ==1]

In [ ]:

assets = results.index.levels[1].unique()  
pricing = get_pricing(np.append(assets, [symbols("SPY")]), start, end + pd.Timedelta(days=30), fields="open_price")  


In [ ]:
results = results.reset_index()

In [ ]:
df_DarvasBoxBreakUp = results[results["DarvasBoxBreakUp"] == 1]

In [ ]:
df_DarvasBoxBreakUp#.iterrows().next()


In [ ]:
#df_DarvasBoxBreakUp.iloc[0]["level_1"]
pricing_return = pricing.pct_change().shift(-1)
pricing_return5 = pricing.pct_change(5).shift(-5)
pricing_return10 = pricing.pct_change(10).shift(-10)


In [ ]:
l = list()
spy = symbols("SPY")
for i, row in df_DarvasBoxBreakUp.iterrows():
    l.append([pricing_return.loc[row["level_0"], row["level_1"],], 
              pricing_return5.loc[row["level_0"], row["level_1"],], 
              pricing_return10.loc[row["level_0"], row["level_1"],],
             pricing_return.loc[row["level_0"], spy,], 
              pricing_return5.loc[row["level_0"], spy,], 
              pricing_return10.loc[row["level_0"], spy,]
             
             ], 
             
              )
    

In [ ]:
pd.DataFrame(l,columns=["return1", "return5", "return10","spyreturn1", "spyreturn5", "spyreturn10", ] ).cumsum().plot()

In [ ]:
pricing_return

In [ ]:
l = list()
for i, row in results.iterrows():
    if row["DarvasBoxBreakUp"] > 0:
        l.append(pricing_return.loc[row["level_0"], row["level_1"]],)
    else:
        l.append(0)
        

In [ ]:
results["return1"] = l

In [ ]:
df_DarvasBoxBreakUp.groupby(by="level_0")["return1"].count().rolling(10).sum().plot()